<a href="https://colab.research.google.com/github/bharatbajoria/Summer-Internship/blob/master/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def statistical_signficance(statistics,stats,stdev,topic_df_T,alpha,null_hyp,model1,np,pd):
  #topics_df_T: Dataframe of words for any model
  #col: Mtx with Topic Header, col[i]= Topic-i
  #alpha: Critical p-Value for rejecting null hypothesis
  #null_hyp: Value of mean for Null Hypothesis
  col=[]
  a='Topic- '
  for i in range(len(topic_df_T.columns)):
    col.append(a+str(i+1))

  
  Average_comparison=null_hyp
  ds=[]
  for i in range((len(col))) :
    ds.append(list(topic_df_T[col[i]]))

  ds# ds[i] is i-th list of Topic-i

  simscore_topic=[]#initiating mtx for sim scores/cosine scores


  for p in range(len(ds)):
    k=len(ds[p])
    o=0

    sim=[]
    simt=[]
  
    for i in range(k):
      simt=[]
      for j in range(k)  :
        o=model1.similarity(ds[p][i],ds[p][j])
        simt.append(o)

      sim.append(simt)

    simscore_topic.append(sim)
# 3-D list, simscore_topic[i] is similarity scores for topic-i
# simscore_topic[i][j] is list of similarity score of j-th word in i-th Topic with other words


  
  t_score_all=[]
  p_value_all=[]
  mean=0
  sd=0

  t_value=0
  dup_score=[]
  p=0
  p_value=[]
  t2=0
  for i in range(len(ds)):
    
    t=[]
    p_value_topic=[]
    for j in range(len(ds[i])):         
      dup_score=simscore_topic[i][j][:]    
      dup_score.pop(j)# removing score of i-th element with itself which is=1
      t2,p=stats.ttest_1samp(np.array(dup_score),popmean=np.array(Average_comparison))
      
      t.append(t2)
      p_value_topic.append(p)
      pass

    t_score_all.append(t)
    p_value_all.append(p_value_topic)
  
  t=0
  word_to_remove=[] #initializing tuple to save topicid and word outside cluster
  for i in range(len(p_value_all)):
    word=[]
    for j in range(len(p_value_all[i])):
      if (p_value_all[i][j]>= alpha) or(t_score_all[i][j]<0):
        word.append(ds[i][j])
    
    if len(word)>0:
      t+=len(word)#counting number of words to be removed
    
    word_to_remove.append(word)


# word_to_remove[i] list of words to remove from topic-i
#ds[i] list of words in topic-i
#t number of words to be removed in a algorithm
# simscore_topic[i][j] is list of similarity score of j-th word in i-th Topic with other words

  
  return word_to_remove,ds,t

In [ ]:
def best_model(number_words_removed_lsi, number_words_removed_hdp, number_words_removed_lda,lda,lsi,hdp):
  
  Best_model=[number_words_removed_lsi, number_words_removed_hdp, number_words_removed_lda]
  k=min(Best_model)
  if k==number_words_removed_lsi:    
    ideal_model=lsi
  if k==number_words_removed_hdp:    
    ideal_model=hdp
  if k==number_words_removed_lda:    
    ideal_model=lda
  
  
  return ideal_model


In [ ]:
def new_topic(ideal_model,dictionary,new_token):
  lda1=ideal_model
  tk=[]
  for i in new_token:
    tk.extend(i)

  new_topic=lda1[dictionary.doc2bow(tk)]
  New_Topic_id,New_Topic_Prob= [],[]
  a='Topic '
  for i in new_topic:    
    a='Topic:'+ str(i[0]+1)    
    New_Topic_id.append(a)
    New_Topic_Prob.append((i[1]*100))

  return New_Topic_id,New_Topic_Prob